In [43]:
#import libraries
import requests
import pandas as pd
import time
import psycopg2 as ps
from dotenv import load_dotenv
import os

In [44]:
load_dotenv()

True

In [45]:
#Keys
API_KEY = os.getenv('API_KEY')
CHANNEL_ID = "UCCT_-OGW5IiUuuHwmuyUPYQ"

In [46]:
def get_video_details(video_id):
    # Collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id=" + video_id + "&part=statistics&key=" + API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    statistics = response_video_stats['items'][0]['statistics']

    # Check if each statistic exists before accessing it
    view_count = statistics.get('viewCount', 0)  # Default to 0 if not found
    like_count = statistics.get('likeCount', 0)  # Default to 0 if not found
    dislike_count = statistics.get('dislikeCount', 0)  # Default to 0 if not found
    comment_count = statistics.get('commentCount', 0)  # Default to 0 if not found

    return view_count, like_count, dislike_count, comment_count


In [47]:
def get_videos(df, api_key=API_KEY, channel_id=CHANNEL_ID):
    # Initialize pageToken
    pageToken = ""
    rows_to_concat = []

    while True:
        # Update URL to include pageToken
        url = f"https://www.googleapis.com/youtube/v3/search?key={api_key}&channelId={channel_id}&part=snippet,id&order=date&maxResults=50&pageToken={pageToken}"
        response = requests.get(url).json()

        # Check if API call was successful
        if response.get('error'):
            print(f"Error fetching data: {response.get('error')}")
            break

        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title'].replace("&amp;", "&")
                upload_date = str(video['snippet']['publishedAt']).split("T")[0]

                view_count, like_count, dislike_count, comment_count = get_video_details(video_id)

                row_dict = {
                    'video_id': video_id, 'video_title': video_title,
                    'upload_date': upload_date, 'view_count': view_count,
                    'like_count': like_count, 'dislike_count': dislike_count,
                    'comment_count': comment_count
                }
                rows_to_concat.append(row_dict)

        df_new_rows = pd.DataFrame(rows_to_concat)
        df = pd.concat([df, df_new_rows], ignore_index=True)

        # Handle pagination
        if 'nextPageToken' in response:
            pageToken = response['nextPageToken']
            time.sleep(1)  # Respect YouTube API rate limits
        else:
            break  # Exit the loop if there's no next page

    return df



In [48]:
#main

#build our dataframe
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","dislike_count","comment_count"])
df = get_videos(df)

In [49]:
df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,2GPKYoU-kUI,The Bracelet that&#39;s Changing the World 🌎#p...,2024-03-20,63,22,0,0
1,Dc5yzM0Ye7Q,Your Friendly Neighborhood Ocean Cleanup Crew ...,2024-03-16,2606,362,0,34
2,nHFmurarFIc,It&#39;s more than just a bracelet...😎🌊 #plast...,2024-03-14,3194,340,0,5
3,moAw-axibqg,35 MILLION POUNDS!!! Time To Celebrate♻️🎉🥳 #pl...,2024-03-12,1287,370,0,23
4,PU5vG46H4DY,The Harmful Effects of Ocean Pollution 😧🌊 #pla...,2024-03-11,6718,464,0,17


In [50]:
# Assuming 'df' is your pandas DataFrame
df.to_csv(r"C:\Users\Levin\Projects\4OceanSA.csv", index=False)


In [51]:
df = pd.read_csv('4OceanSA.csv')
df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,2GPKYoU-kUI,The Bracelet that&#39;s Changing the World 🌎#p...,2024-03-20,63,22,0,0
1,Dc5yzM0Ye7Q,Your Friendly Neighborhood Ocean Cleanup Crew ...,2024-03-16,2606,362,0,34
2,nHFmurarFIc,It&#39;s more than just a bracelet...😎🌊 #plast...,2024-03-14,3194,340,0,5
3,moAw-axibqg,35 MILLION POUNDS!!! Time To Celebrate♻️🎉🥳 #pl...,2024-03-12,1287,370,0,23
4,PU5vG46H4DY,The Harmful Effects of Ocean Pollution 😧🌊 #pla...,2024-03-11,6718,464,0,17


In [52]:
#connect to db
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    except ps.OperationalError as e:
            raise e
    else:
        print('Connected!')
    return conn

In [53]:
host_name = os.getenv('host_name')
dbname = os.getenv('dbname')
port = os.getenv('port')
username = os.getenv('un')
password = os.getenv('password')
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [54]:
#create table
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                     video_id VARCHAR(255) PRIMARY KEY,
                     video_title TEXT NOT NULL,
                     upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                     view_count INTEGER NOT NULL,
                     like_count INTEGER NOT NULL,
                     dislike_count INTEGER NOT NULL,
                     comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

In [55]:
#check if video exists
def check_if_video_exists(curr, video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query, (video_id,))

    return curr.fetchone() is not None

In [56]:
#update row if video exists

def update_row(curr, video_id, video_title, view_count, like_count, dislike_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                dislike_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, dislike_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [57]:
def update_db(curr, df):
    # Initialize an empty list to store rows for new videos
    new_videos = []

    for video_id, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']):  # if video already exists then we will update
            update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['dislike_count'], row['comment_count'])
        else:  # The video doesn't exist so we will collect rows to append to the db table later
            # Convert the Series to a DataFrame row and set 'video_id' as a column
            new_row_df = row.to_frame().T
            new_row_df['video_id'] = row['video_id']
            new_videos.append(new_row_df)

    # If there are new videos, concatenate them into a single DataFrame
    if new_videos:
        tmp_df = pd.concat(new_videos, ignore_index=True)
    else:
        # If there are no new videos, create an empty DataFrame with the specified columns
        tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'dislike_count', 'comment_count'])

    return tmp_df


In [58]:
#write insert command

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (video_id) DO NOTHING;""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [59]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['dislike_count'], row['comment_count'])


In [60]:
curr = conn.cursor()

In [61]:
create_table(curr)

In [62]:
new_vid_df = update_db(curr, df)

In [63]:
append_from_df_to_db(curr, new_vid_df)

In [64]:
conn.commit()